<a href="https://colab.research.google.com/github/Sallb679/UMD-Professor-Ratings-Analysis/blob/main/HW4_CMSC_320_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook i will scrape data from https://planetterp.com/ to do an analysis on professor reviews and see if i can predict their star rating

some feautures I am considering:
number of classes they teach
experience (how many years/semesters they taught)
total number of reviews
sentiment of the reviews
expected_grade
time


Msking table schema

table 1 professors
table 2 courses
table 3 grades

In [ ]:
# STEP 1: Imports
import requests
import pandas as pd
import time
from tqdm import tqdm

# STEP 2: Scraper Class
class PlanetTerpScraper:
    def __init__(self):
        self.headers = {"User-Agent": "Mozilla/5.0"}
        self.professors = []
        self.reviews = []
        self.grades = []

    def get_professors_batch(self, limit=100, offset=0):
        url = f"https://planetterp.com/api/v1/professors?limit={limit}&offset={offset}"
        res = requests.get(url, headers=self.headers)
        return res.json()

    def get_reviews_for_prof(self, name):
        url = f"https://api.planetterp.com/v1/professor?name={name}&reviews=true"
        try:
            res = requests.get(url, headers=self.headers)
            data = res.json()
            for r in data.get("reviews", []):
                r["professor"] = name
            return data.get("reviews", [])
        except:
            return []

    def get_grades_for_prof_course(self, course, professor):
        url = f"https://api.planetterp.com/v1/grades?course={course}&professor={professor}"
        try:
            res = requests.get(url, headers=self.headers)
            data = res.json()
            if "error" in data:
                return []
            return data
        except:
            return []

    def fetch_all_professors(self, total=13425, batch_size=100):
        for offset in tqdm(range(0, total, batch_size)):
            batch = self.get_professors_batch(limit=batch_size, offset=offset)
            if not batch:
                break
            self.professors.extend(batch)
            time.sleep(0.2)
        return pd.DataFrame(self.professors)

    def fetch_all_reviews(self):
        for prof in tqdm(self.professors):
            name = prof["name"]
            self.reviews.extend(self.get_reviews_for_prof(name))
            time.sleep(0.5)
        return pd.DataFrame(self.reviews)

    def fetch_professor_grades_from_reviews(self, reviews_df):
        seen = set()
        for _, row in tqdm(reviews_df.iterrows(), total=len(reviews_df)):
            course = row["course"]
            professor = row["professor"]
            if pd.isna(course) or pd.isna(professor):
                continue
            key = (course, professor)
            if key in seen:
                continue
            seen.add(key)
            self.grades.extend(self.get_grades_for_prof_course(course, professor))
            time.sleep(0.25)
        return pd.DataFrame(self.grades)

# STEP 3: Run Scraper End to End
scraper = PlanetTerpScraper()

# 1. Download professors
print("📥 Downloading professors...")
prof_df = scraper.fetch_all_professors()
prof_df.to_csv("professors.csv", index=False)
print(f"✅ Saved {len(prof_df)} professors.")

# 2. Download reviews
print("🗣️ Downloading reviews...")
reviews_df = scraper.fetch_all_reviews()
reviews_df.to_csv("reviews.csv", index=False)
print(f"✅ Saved {len(reviews_df)} reviews.")

# 3. Download professor-specific grades
print("📊 Downloading professor-specific grades...")
grades_df = scraper.fetch_professor_grades_from_reviews(reviews_df)
grades_df.to_csv("grades.csv", index=False)
print(f"✅ Saved {len(grades_df)} grade records.")



📥 Downloading professors...


100%|██████████| 135/135 [02:42<00:00,  1.20s/it]


✅ Saved 13425 professors.
🗣️ Downloading reviews...


100%|██████████| 13425/13425 [2:41:08<00:00,  1.39it/s]


✅ Saved 37401 reviews.
📊 Downloading professor-specific grades...


100%|██████████| 37401/37401 [1:08:03<00:00,  9.16it/s]


✅ Saved 71543 grade records.
